In [11]:
import pandas as pd

In [12]:
ai_result_path = "./AI_Mapping_Result/V3/1_BAY_mapping_result_V3_rev6.csv"
prod_result_path = "./Listed_Mapping_Prod/BAY_Q2_2024.csv"
th_month = [
    'มกราคม', 
    'กุมภาพันธ์', 
    'มีนาคม',
    'เมษายน',
    'พฤษภาคม',
    'มิถุนายน',
    'กรกฎาคม',
    'สิงหาคม',
    'กันยายน',
    'ตุลาคม',
    'พฤศจิกายน',
    'ธันวาคม',
]

th_month_short = [
    'ม.ค.', 
    'ก.พ.', 
    'มี.ค.',
    'เม.ย.',
    'พ.ค.',
    'มิ.ย.',
    'ก.ค.',
    'ส.ค.',
    'ก.ย.',
    'ต.ค.',
    'พ.ย.',
    'ธ.ค.',
]

In [13]:
def replace_comma(name):
    return name.replace(",", "")

def replace_month(name):
    for month in th_month:
        if (month in name):
            name = name.replace(month, "")

    for month in th_month_short:
        if (month in name):
            name = name.replace(month, "")
    return name

In [14]:
dtype = {
    'query': 'string',
    'set_code': 'string'
}

ai_df = pd.read_csv(ai_result_path, encoding='utf-8', dtype=dtype)

ai_df_query = ai_df['query']
ai_df_query.replace(to_replace=' ', value='', regex=True, inplace=True)
ai_df_query.replace(to_replace=r'[0-9]', value='', regex=True, inplace=True)
ai_df['query'] = ai_df_query

ai_df_query = ai_df['query']
for index, row in ai_df_query.items():
    name = row
    name_replace = replace_month(name)
    name_replace = replace_comma(name_replace)
    ai_df['query'].iloc[index] = name_replace

ai_df.iloc[21]

Unnamed: 0                                                         21
query                        ทุนจดทะเบียนหุ้นสามัญหุ้นมูลค่าหุ้นละบาท
set_code                                                       310120
item_in_taxonomy                      ทุนจดทะเบียน หุ้นสามัญจดทะเบียน
reasoning           The input mentions ทุนจดทะเบียน and หุ้นสามัญ,...
table_name                                              งบฐานะการเงิน
category                                     หนี้สินและส่วนของเจ้าของ
number_values                [75741437, 75741437, 75741437, 75741437]
Name: 21, dtype: object

In [15]:
dtype = {
    'account_name': 'string',
    'account_code': 'string',
}

prod_df = pd.read_csv(prod_result_path, encoding='utf-8', dtype=dtype)
prod_acc_name = prod_df['account_name']
prod_acc_name.replace(to_replace=r'\{{\w+\}}', value='', regex=True, inplace=True)
prod_df['account_name'] = prod_acc_name
prod_df.iloc[22]

account_name                      ทุนจดทะเบียนหุ้นสามัญหุ้นมูลค่าหุ้นละบาท
taxonomy_account_name    ทุนจดทะเบียนหุ้นสามัญ{{number}}หุ้นมูลค่าหุ้นล...
account_code                                                        310120
values                                        ['75,741,437', '75,741,437']
Name: 22, dtype: object

In [16]:
compared_list = []

for index, row in prod_df.iterrows():
    prod_taxo_name = row['taxonomy_account_name']
    prod_acc_code = row['account_code']
    prod_acc_name = row['account_name']
    # print(prod_taxo_name)
    # print(prod_acc_code, type(prod_acc_code))

    ai_row = ai_df[(ai_df['query'] == prod_acc_name) & (ai_df['set_code'] == prod_acc_code)]
    # print(len(ai_row.index))

    if (len(ai_row.index) > 1):
        print(prod_acc_name)
        print(len(ai_row.index))
        print()

    ai_acc_code = ""
    ai_acc_name = ""
    if (len(ai_row.index) > 0):
        ai_acc_code = ai_row['set_code'].iloc[0]
        ai_acc_name = ai_row['query'].iloc[0]
    # print(ai_acc_code)

    compare_item = {
        "taxonomy_account_name": prod_taxo_name,
        "ai_account_code": ai_acc_code,
        "ai_acc_name": ai_acc_name,
        "prod_account_name": prod_acc_name,
        "prod_account_code": prod_acc_code,
        "is_match": str(ai_acc_code) == str(prod_acc_code)
    }

    # print(compare_item)
    # print(compare_item['taxonomy_account_name'], compare_item['ai_account_code'], compare_item['ai_acc_name'], compare_item['prod_account_name'], compare_item['prod_account_code'], compare_item['is_match'])
    compared_list.append(compare_item)

ทุนจดทะเบียนหุ้นสามัญหุ้นมูลค่าหุ้นละบาท
2

ทุนที่ออกและชำระแล้วหุ้นสามัญหุ้นมูลค่าหุ้นละบาท
2

รายได้ดอกเบี้ย
2

ค่าใช้จ่ายดอกเบี้ย
2

รายได้ดอกเบี้ยสุทธิ
2

รายได้ค่าธรรมเนียมและบริการ
2

ค่าใช้จ่ายค่าธรรมเนียมและบริการ
2

รายได้ค่าธรรมเนียมและบริการสุทธิ
2

กำไรสุทธิจากเครื่องมือทางการเงินที่วัดมูลค่าด้วยมูลค่ายุติธรรมผ่านกำไรหรือขาดทุน
2

ส่วนแบ่งกำไรจากเงินลงทุนตามวิธีส่วนได้เสีย
2

รายได้จากเงินปันผล
2

รายได้จากการดำเนินงานอื่นๆ
2

ค่าใช้จ่ายเกี่ยวกับพนักงาน
2

ค่าตอบแทนกรรมการ
2

ค่าใช้จ่ายเกี่ยวกับอาคารสถานที่และอุปกรณ์
2

ค่าภาษีอากร
2

อื่นๆ
2

ผลขาดทุนด้านเครดิตที่คาดว่าจะเกิดขึ้น
2

กำไรจากการดำเนินงานก่อนภาษีเงินได้
2

ภาษีเงินได้
2

กำไรสุทธิ
2

กำไร(ขาดทุน)จากการวัดมูลค่าเงินลงทุนในตราสารหนี้ด้วยมูลค่ายุติธรรมผ่านกำไรขาดทุนเบ็ดเสร็จอื่น
2

กำไร(ขาดทุน)จากการวัดมูลค่ายุติธรรมเครื่องมือที่ใช้สำหรับการป้องกันความเสี่ยง
2

กำไร(ขาดทุน)จากการแปลงค่างบการเงินจากการดำเนินงานในต่างประเทศ
2

ภาษีเงินได้เกี่ยวกับองค์ประกอบของกำไร(ขาดทุน)เบ็ดเสร็จอื่น
3

กำไร(ขาดทุน)จากเงินลงทุนที

In [17]:
count_unmatch = 0
for item in compared_list:
    if (item['is_match'] == False):
        count_unmatch += 1
        print(item)

print(f"Total Unmatch: {count_unmatch}")

{'taxonomy_account_name': 'สินทรัพย์ทางการเงินที่วัดมูลค่าด้วยมูลค่ายุติธรรมผ่านกำไรหรือขาดทุน', 'ai_account_code': '', 'ai_acc_name': '', 'prod_account_name': 'สินทรัพย์ทางการเงินที่วัดมูลค่าด้วยมูลค่ายุติธรรมผ่านกำไรหรือขาดทุน', 'prod_account_code': '100690', 'is_match': False}
{'taxonomy_account_name': 'รวมสินทรัพย์', 'ai_account_code': '', 'ai_acc_name': '', 'prod_account_name': 'รวมสินทรัพย์', 'prod_account_code': '109999', 'is_match': False}
{'taxonomy_account_name': 'ตราสารหนี้ที่ออกและเงินกู้ยืม', 'ai_account_code': '', 'ai_acc_name': '', 'prod_account_name': 'ตราสารหนี้ที่ออกและเงินกู้ยืม', 'prod_account_code': '202090', 'is_match': False}
{'taxonomy_account_name': 'องค์ประกอบอื่นของส่วนของเจ้าของ', 'ai_account_code': '', 'ai_acc_name': '', 'prod_account_name': 'องค์ประกอบอื่นของส่วนของเจ้าของ', 'prod_account_code': '319990', 'is_match': False}
{'taxonomy_account_name': 'ทุนสำรองตามกฎหมาย', 'ai_account_code': '', 'ai_acc_name': '', 'prod_account_name': 'ทุนสำรองตามกฎหมาย', 'pr

In [20]:
no_code_count = 0
for item in compared_list:
    if (item['is_match'] == False):
        if (item['ai_account_code'] == ''):
            no_code_count += 1
            print(item['prod_account_code'])

print(f"No Code From AI: {no_code_count}")

100690
109999
202090
319990
310611
309998
450600
450600
510200
510400
510800
512300
512300
514800
511510
512000
514920
514710
514900
516900
517000
519999
529999
539999
No Code From AI: 24
